### Instructions


##### Creating Catalog and Schemas

In [0]:
%sql
-- Creating Catalog
CREATE CATALOG IF NOT EXISTS db_Academy
MANAGED LOCATION 'abfss://unity-catalog-storage@dbstoragertkgqhhy66nu6.dfs.core.windows.net/1290381263006333'

In [0]:
%sql
create schema if not exists db_academy.production

In [0]:
%sql
show schemas in db_academy

In [0]:
%sql
use catalog db_academy;
use schema production;

In [0]:
%sql
SELECT 
  current_catalog(), 
  current_schema()

In [0]:
%sql
describe schema extended production

##### Data Source</br>
Volumes has been downloaded from:
[Link to Data Source](https://adb-1290381263006333.13.azuredatabricks.net/marketplace/consumer/listings/ca87a8be-9a17-4469-acee-d992d6cd16ca?o=1290381263006333) </br>
Saved as db_academy_ecomerce under group "Delta Shares Received"

In [0]:
%sql
describe volume db_academy_ecomerce.v01.raw

In [0]:
# exploring Volumes within Catalog
spark.sql("LIST '/Volumes/db_academy_ecomerce/v01/raw/users-historical/'").display()

### Lakeflow Connect